# Skill extraction

In [18]:
import sys
import os
import pandas as pd

# Get the project root directory (e.g., linkedin_jobs_analysis/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

from src.utils.logger import setup_logging
from src.analysis.extracting_skills_list import SkillExtractor
from config.analysis import STANDARD_SKILL_MAP

setup_logging()

In [29]:
def test_skill_extractor(test_cases = [
        (
            "Precisamos de um profissional com experiência em Python e SQL.",
            ['Python', 'SQL']
        ),
        (
            "Conhecimento em Machine Learning (ML) e PowerBI é essencial.",
            ['Machine Learning', 'Power BI']
        ),
        (
            "Domínio de AWS, Azure e GCP para cloud computing.",
            ['Cloud', 'AWS', 'Azure', 'GCP']
        ),
        (
            "Experiência com PySpark e Spark para processamento de dados.",
            ['Spark']
        ),
        (
            "Habilidade em Excel avançado e análise de dados.",
            ['Excel']
        ),
        (
            "Conhecimento em Airflow para orquestração de pipelines.",
            ['Airflow']
        ),
        (
            "Não há requisitos técnicos específicos para esta vaga.",
            []
        ),
        (
            "Necessário conhecimento em Python e pandas para análise de dados.",
            ['Python', 'Pandas']
        ),
        (
            "A palavra 'excelente' não deve ser confundida com 3xc3l.",
            []
        ),
        (
            "MLOps e Machine Learning são importantes para a vaga.",
            ['Machine Learning']  # Assuming MLOps isn't in our test skills
        )
    ]):
    """Test function for SkillExtractor with various edge cases"""

    extractor = SkillExtractor(STANDARD_SKILL_MAP)
    
    passed = 0
    failed = 0
    failed_cases = []
    
    for text, expected in test_cases:
        try:
            result = extractor.extract_skills(text)
            result_set = set(result)
            expected_set = set(expected)
            
            # Check for missing skills
            missing = expected_set - result_set
            # Check for extra skills (only fail if we got completely unexpected skills)
            extra = result_set - expected_set
            print(extra)
            if not missing and not extra:
                passed += 1
                print(f"PASS: '{text}' -> {result}")
            else:
                failed += 1
                failed_cases.append({
                    'text': text,
                    'expected': expected,
                    'got': result,
                    'missing': list(missing),
                    'extra': list(extra)
                })
                print(f"FAIL: '{text}'")
                print(f"  Expected: {expected}")
                print(f"  Got:      {result}")
                if missing:
                    print(f"  Missing:  {list(missing)}")
                if extra:
                    print(f"  Extra:    {list(extra)}")
                    
        except Exception as e:
            failed += 1
            failed_cases.append({
                'text': text,
                'error': str(e)
            })
            print(f"ERROR processing: '{text}' - {str(e)}")
    
    # Print summary
    print(f"\nTest Results: {passed} passed, {failed} failed")
    
    if failed_cases:
        print("\nFailed Cases Summary:")
        for case in failed_cases:
            if 'error' in case:
                print(f"Text: '{case['text']}'")
                print(f"Error: {case['error']}")
            else:
                print(f"Text: '{case['text']}'")
                print(f"Expected: {case['expected']}")
                print(f"Got:      {case['got']}")
                if case['missing']:
                    print(f"Missing:  {case['missing']}")
                if case['extra']:
                    print(f"Extra:    {case['extra']}")
            print("---")
    
    return {
        'passed': passed,
        'failed': failed,
        'failed_cases': failed_cases,
        'success_rate': passed / (passed + failed) if (passed + failed) > 0 else 0
    }

In [34]:
# skills_list = [skill for synonyms in STANDARD_SKILL_MAP.values() for skill in synonyms]
extractor = SkillExtractor(STANDARD_SKILL_MAP)
text = 'MLOps são importantes para a vaga.'
extractor.extract_skills(text)

2025-05-18 17:30:52,946 - src.analysis.extracting_skills_list - SUCCESS - Successfully loaded spaCy model: pt_core_news_lg
2025-05-18 17:30:52,948 - src.analysis.extracting_skills_list - INFO - Prepared regex patterns for 126 skills.


['MLOps']

In [31]:
test_skill_extractor()

2025-05-18 17:30:30,647 - src.analysis.extracting_skills_list - SUCCESS - Successfully loaded spaCy model: pt_core_news_lg
2025-05-18 17:30:30,648 - src.analysis.extracting_skills_list - INFO - Prepared regex patterns for 126 skills.


set()
PASS: 'Precisamos de um profissional com experiência em Python e SQL.' -> ['SQL', 'Python']
set()
PASS: 'Conhecimento em Machine Learning (ML) e PowerBI é essencial.' -> ['Power BI', 'Machine Learning']
set()
PASS: 'Domínio de AWS, Azure e GCP para cloud computing.' -> ['AWS', 'GCP', 'Azure', 'Cloud']
set()
PASS: 'Experiência com PySpark e Spark para processamento de dados.' -> ['Spark']
set()
PASS: 'Habilidade em Excel avançado e análise de dados.' -> ['Excel']
set()
PASS: 'Conhecimento em Airflow para orquestração de pipelines.' -> ['Airflow']
set()
PASS: 'Não há requisitos técnicos específicos para esta vaga.' -> []
set()
PASS: 'Necessário conhecimento em Python e pandas para análise de dados.' -> ['Pandas', 'Python']
set()
PASS: 'A palavra 'excelente' não deve ser confundida com 3xc3l.' -> []
{'MLOps'}
FAIL: 'MLOps e Machine Learning são importantes para a vaga.'
  Expected: ['Machine Learning']
  Got:      ['Machine Learning', 'MLOps']
  Extra:    ['MLOps']

Test Results: 9 

{'passed': 9,
 'failed': 1,
 'failed_cases': [{'text': 'MLOps e Machine Learning são importantes para a vaga.',
   'expected': ['Machine Learning'],
   'got': ['Machine Learning', 'MLOps'],
   'missing': [],
   'extra': ['MLOps']}],
 'success_rate': 0.9}

In [15]:
jobs = pd.read_csv('../data/processed/df_jobs_classified.csv')
skills = pd.read_csv('../data/processed/df_skills.csv')

---

# Titles

In [1]:
import re
from typing import Dict, List
import pandas as pd


In [1]:
import sys
import os

# Get the project root directory (e.g., linkedin_jobs_analysis/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

# Now import from src
from config.analysis import ROLE_PATTERNS, SPECIAL_CASES
from src.analysis.analysis_utils import test_classifier, title_classifier


In [2]:
test_classifier()



Test results: 24 passed, 0 failed out of 24 cases.


AttributeError: 'Logger' object has no attribute 'success'

In [4]:
title_classifier('Analista de dados e automação com IA - Pleno	')

'Analista de Dados'

In [5]:
jobs_classified = pd.read_csv('jobs_classified.csv')
jobs_classified_tuple = list(zip(jobs_classified['job_title'], jobs_classified['classified_job_title']))

test_classifier(jobs_classified_tuple)

FAIL: 'Analista de dados e automação com IA - Pleno'
  Expected: Engenheiro de IA
  Got:      Analista de Dados
FAIL: 'Engenheiro (a) de IA'
  Expected: Outros
  Got:      Engenheiro de IA
FAIL: 'Engenheiro (a) de Automação Sr / CS / Uberlândia - MG'
  Expected: Engenheiro de IA
  Got:      Outros
FAIL: 'Engenheiro (a) de IA'
  Expected: Outros
  Got:      Engenheiro de IA
FAIL: 'Engenheiro (a) de IA'
  Expected: Outros
  Got:      Engenheiro de IA
FAIL: 'Engenheiro (a) de IA'
  Expected: Outros
  Got:      Engenheiro de IA
FAIL: 'Engenheiro(a) de Software (Fullstack) | IT Energy'
  Expected: Outros
  Got:      Engenheiro de Software
FAIL: 'Engenheiro(a) de Software Pleno – Plataforma de IA'
  Expected: Outros
  Got:      Engenheiro de Software
FAIL: 'Engenheiro de Automação Industrial'
  Expected: Engenheiro de IA
  Got:      Outros
FAIL: 'Engenheiro(a) de Software Frontend'
  Expected: Outros
  Got:      Engenheiro de Software
FAIL: 'Engenheiro(a) de Software (.NET) Jr/Pleno | Electr

In [27]:
ROLE_PATTERNS['Cientista de Dados']

['\\b(data scientist|cientista de dados)\\b',
 '\\b(ml|machine learning|deep learning|llm|nlp)\\b',
 '(pesquisador|research).*(dados|data)',
 'ci[êe]ncia de dados',
 '\\bgenai\\b',
 '(computer vision|visão computacional)']

In [7]:
jobs_classified[jobs_classified['classified_job_title']=='Engenheiro de Dados'].sample(20)

,Unnamed: 0,job_title,classified_job_title
2752,2752,Azure Data Engineer,Engenheiro de Dados
3059,3059,Blockchain Data Engineer (Senior/Lead) ID34521,Engenheiro de Dados
3035,3035,Data Engineer with PowerCenter Experience - Re...,Engenheiro de Dados
1076,1076,Engenheiro(a) de Dados (DevOps/DataOps),Engenheiro de Dados
737,737,Engenheiro de Dados - Trabalho Remoto | REF#25...,Engenheiro de Dados
1886,1886,Engenheiro/ Engenheira de Dados Pyspark,Engenheiro de Dados
833,833,Engenheiro(a) de Dados,Engenheiro de Dados
1911,1911,Engenheiro de dados,Engenheiro de Dados
2398,2398,Data Engineer,Engenheiro de Dados
2893,2893,Banco de Talentos Data Engineer,Engenheiro de Dados


---

# Location

In [4]:
import pandas as pd
import re


In [19]:
def standardize_locations(df: pd.DataFrame, location_col: str = 'location') -> pd.DataFrame:
    """
    Standardizes location data into separate city, state, and country columns.
    Handles cases like "São Paulo, Brasil" and "Distrito Federal, Brasil" correctly.
    Also handles "e Região" patterns for state capitals.
    """
    
    df['city'] = None
    df['state'] = None
    df['country'] = 'Brasil'
    
    brazilian_states = {
        'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapá', 'AM': 'Amazonas',
        'BA': 'Bahia', 'CE': 'Ceará', 'DF': 'Distrito Federal',
        'ES': 'Espírito Santo', 'GO': 'Goiás', 'MA': 'Maranhão',
        'MT': 'Mato Grosso', 'MS': 'Mato Grosso do Sul',
        'MG': 'Minas Gerais', 'PA': 'Pará', 'PB': 'Paraíba',
        'PR': 'Paraná', 'PE': 'Pernambuco', 'PI': 'Piauí',
        'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte',
        'RS': 'Rio Grande do Sul', 'RO': 'Rondônia',
        'RR': 'Roraima', 'SC': 'Santa Catarina',
        'SP': 'São Paulo', 'SE': 'Sergipe', 'TO': 'Tocantins'
    }
    
    state_capitals = {
        'Rio Branco': 'AC', 'Maceió': 'AL', 'Macapá': 'AP', 'Manaus': 'AM',
        'Salvador': 'BA', 'Fortaleza': 'CE', 'Brasília': 'DF', 'Vitória': 'ES',
        'Goiânia': 'GO', 'São Luís': 'MA', 'Cuiabá': 'MT', 'Campo Grande': 'MS',
        'Belo Horizonte': 'MG', 'Belém': 'PA', 'João Pessoa': 'PB', 'Curitiba': 'PR',
        'Recife': 'PE', 'Teresina': 'PI', 'Rio De Janeiro': 'RJ', 'Natal': 'RN',
        'Porto Alegre': 'RS', 'Porto Velho': 'RO', 'Boa Vista': 'RR', 'Florianópolis': 'SC',
        'São Paulo': 'SP', 'Aracaju': 'SE', 'Palmas': 'TO'
    }
    
    state_mapping = {**{v: k for k, v in brazilian_states.items()}, **brazilian_states}
    
    state_names = set(brazilian_states.values())
    state_abbrevs = set(brazilian_states.keys())
    
    def extract_location(location):
        if pd.isna(location):
            return (None, None, None)
            
        location = str(location).strip()

        if location.lower() in ['brasil']:
            return (None, None, 'Brasil')
        
        if location in state_names or location in state_abbrevs:
            state_code = location if location in state_abbrevs else state_mapping.get(location)
            return (None, state_code, 'Brasil')
            
        match = re.match(r'^(?P<state>[^,]+),\s*Brasil$', location, re.IGNORECASE)
        if match and match.group('state') in state_names:
            state_name = match.group('state')
            return (None, state_mapping.get(state_name), 'Brasil')
        
        match = re.match(r'^(?P<city>.+)\s+e\s+Região$', location)
        if match:
            city = match.group('city').strip().title()
            state = state_capitals.get(city)
            return (city, state, 'Brasil')
        
        match = re.match(r'^(?P<city>[^,]+),\s*(?P<state>[A-Z]{2})$', location)
        if match:
            return (match.group('city').title(), match.group('state').upper(), 'Brasil')
        
        match = re.match(r'^(?P<city>[^,]+),\s*(?P<state>.+)$', location)
        if match:
            state = match.group('state').strip()
            
            if state.lower() in ['brasil', 'brazil']:
                city_name = match.group('city').title()
                return (city_name, state_capitals.get(city_name), 'Brasil')
                
            if state in state_names or state in state_abbrevs:
                state_code = state if state in state_abbrevs else state_mapping.get(state)
                return (match.group('city').title(), state_code, 'Brasil')
            
            return (match.group('city').title(), None, 'Brasil')
        
        city_name = location.title()
        if city_name in state_capitals:
            return (city_name, state_capitals.get(city_name), 'Brasil')
        
        return (city_name, None, 'Brasil')
    
    df[['city', 'state', 'country']] = df[location_col].apply(
        lambda x: pd.Series(extract_location(x))
    )
    
    return df

In [20]:
brazilian_states = {
    'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapá', 'AM': 'Amazonas',
    'BA': 'Bahia', 'CE': 'Ceará', 'DF': 'Distrito Federal',
    'ES': 'Espírito Santo', 'GO': 'Goiás', 'MA': 'Maranhão',
    'MT': 'Mato Grosso', 'MS': 'Mato Grosso Do Sul',
    'MG': 'Minas Gerais', 'PA': 'Pará', 'PB': 'Paraíba',
    'PR': 'Paraná', 'PE': 'Pernambuco', 'PI': 'Piauí',
    'RJ': 'RioDe Janeiro', 'RN': 'Rio Grande Do Norte',
    'RS': 'Rio Grande Do Sul', 'RO': 'Rondônia',
    'RR': 'Roraima', 'SC': 'Santa Catarina',
    'SP': 'São Paulo', 'SE': 'Sergipe', 'TO': 'Tocantins'
}

# Brazilian state capitals mapping (capital city: state abbreviation)
state_capitals = {
    'Rio Branco': 'AC', 'Maceió': 'AL', 'Macapá': 'AP', 'Manaus': 'AM',
    'Salvador': 'BA', 'Fortaleza': 'CE', 'Brasília': 'DF', 'Vitória': 'ES',
    'Goiânia': 'GO', 'São Luís': 'MA', 'Cuiabá': 'MT', 'Campo Grande': 'MS',
    'Belo Horizonte': 'MG', 'Belém': 'PA', 'João Pessoa': 'PB', 'Curitiba': 'PR',
    'Recife': 'PE', 'Teresina': 'PI', 'Rio De Janeiro': 'RJ', 'Natal': 'RN',
    'Porto Alegre': 'RS', 'Porto Velho': 'RO', 'Boa Vista': 'RR', 'Florianópolis': 'SC',
    'São Paulo': 'SP', 'Aracaju': 'SE', 'Palmas': 'TO'
}

# Create bidirectional mapping for state names and abbreviations
state_mapping = {**{v: k for k, v in brazilian_states.items()}, **brazilian_states}

    # Set of state names (to check if a location is a state)
state_names = set(brazilian_states.values())
state_abbrevs = set(brazilian_states.keys())

def extract_location(location):
    if pd.isna(location):
        return (None, None, None)
        
    location = str(location).strip()
    
    # Handle country-level or remote cases
    if location.lower() in ['brasil', 'brazil', 'remote', 'remoto']:
        return (None, None, 'Brasil')
    
    # Handle special case of "Distrito Federal, Brasil" and other direct state references
    if location in state_names or location in state_abbrevs:
        state_code = location if location in state_abbrevs else state_mapping.get(location)
        return (None, state_code, 'Brasil')
        
    # Check for state directly followed by country ("Distrito Federal, Brasil")
    match = re.match(r'^(?P<state>[^,]+),\s*Brasil$', location, re.IGNORECASE)
    if match and match.group('state') in state_names:
        state_name = match.group('state')
        return (None, state_mapping.get(state_name), 'Brasil')
    
    # Pattern for "City e Região" (e.g., "São Paulo e Região")
    # Important: What comes with "e Região" is ALWAYS the city
    match = re.match(r'^(?P<city>.+)\s+e\s+Região$', location)
    if match:
        city = match.group('city').strip().title()
        # Get state from the capital mapping
        state = state_capitals.get(city)
        return (city, state, 'Brasil')
    
    # Pattern for "City, ST" (e.g., "São Paulo, SP")
    match = re.match(r'^(?P<city>[^,]+),\s*(?P<state>[A-Z]{2})$', location)
    if match:
        return (match.group('city').title(), match.group('state').upper(), 'Brasil')
    
    # Pattern for "City, State" (e.g., "São Paulo, São Paulo")
    match = re.match(r'^(?P<city>[^,]+),\s*(?P<state>.+)$', location)
    if match:
        state = match.group('state').strip()
        
        # Check if state is actually a country reference
        if state.lower() in ['brasil', 'brazil']:
            city_name = match.group('city').title()
            # Check if city is a state capital
            return (city_name, state_capitals.get(city_name), 'Brasil')
            
        # Check if the supposed state is actually a state
        if state in state_names or state in state_abbrevs:
            state_code = state if state in state_abbrevs else state_mapping.get(state)
            return (match.group('city').title(), state_code, 'Brasil')
        
        # Otherwise assume first part is city, second part unknown
        return (match.group('city').title(), None, 'Brasil')
    
    city_name = location.title()
    # Check if the city is a state capital
    if city_name in state_capitals:
        return (city_name, state_capitals.get(city_name), 'Brasil')
    
    return (city_name, None, 'Brasil')

In [22]:
dict_test = {'location': ['Brasil']}
df = pd.DataFrame(dict_test)
standardize_locations(df, 'location')

,location,city,state,country
0,Brasil,None,None,Brasil


In [7]:
extract_location('Brasil')

(None, None, 'Brasil')

In [69]:
def test_location(test_cases):
    """Test function to verify classifier behavior"""

    failed = 0
    for location, expected in test_cases:
        result = extract_location(location)
        if result != expected:
            print(f"FAIL: '{location}'")
            print(f"  Expected: {expected}")
            print(f"  Got:      {result}")
            failed += 1

    print(f"\nTest results: {len(test_cases)-failed} passed, {failed} failed")

In [70]:
test_cases = [
    ('São Paulo, SP', ('São Paulo', 'SP', 'Brasil')),
    ('Brasil', (None, None, 'Brasil')),
    ('São Paulo e Região', ('São Paulo', 'SP', 'Brasil')),
    ('Rio de Janeiro e Região', ('Rio De Janeiro', 'RJ', 'Brasil')),
    ('Distrito Federal, Brasil', (None, 'DF', 'Brasil'))
    ]

In [71]:
test_location(test_cases)


Test results: 5 passed, 0 failed


In [72]:
jobs = pd.read_csv('../data/processed/df_jobs_classified.csv')
job_copy = jobs.copy()
df = standardize_locations(jobs, location_col='location')

KeyError: 'location'

In [40]:
df[df['location'].str.contains('remoto')]

,job_id,work_model,keyword,scrape_date,job_title,company_name,location,num_applicants,xp_level,job_type,job_sectors,job_description,classified_job_title,post_date,city,state,country


In [50]:
job_copy['city'].unique()

array(['Barueri', 'São Paulo', 'Maracanaú', 'Brasília', 'Curitiba',
       'Guará', 'Blumenau', 'Campinas', 'Fortaleza', 'Indaial',
       'Rio de Janeiro', 'Contagem', 'Caxias do Sul', 'Mogi das Cruzes',
       'Uberlândia', 'Porto Alegre', 'Cravinhos', 'Pereiro', 'Itajaí',
       'Betim', 'Macaé', 'Goiânia', 'Palotina', 'Campo Grande', 'Serra',
       'Tijucas', 'Novo Hamburgo', 'Osasco', 'Maringá', 'Cachoeirinha',
       'Vitória', 'Mossoró', 'Sorocaba', 'Dois Irmãos', 'Arcos',
       'Jaraguá do Sul', 'Joinville', 'Capivari', 'Várzea Grande', None,
       'Marília', 'Aracaju', 'Pato Branco', 'Franca', 'Rio do Sul',
       'Sumaré', 'Ibirama', 'Belo Horizonte', 'Recife', 'Manaus',
       'Embu das Artes', 'Eldorado do Sul', 'Viçosa',
       'São José do Rio Preto', 'Leopoldina', 'Monte Belo', 'Londrina',
       'Salvador', 'Morrinhos', 'Vila Velha', 'Ribeirão Preto', 'Cuiabá',
       'Florianópolis', 'Belém', 'Jundiaí', 'Santo André', 'Santa Rosa',
       'Guarulhos', 'Matão', 'Luca

In [53]:
df['city'].unique().sort()
df['city'].unique()

array(['Barueri', 'São Paulo', 'Maracanaú', 'Brasília', 'Curitiba',
       'Guará', 'Blumenau', 'Campinas', 'Fortaleza', 'Indaial',
       'Rio De Janeiro', 'Contagem', 'Caxias Do Sul', 'Mogi Das Cruzes',
       'Uberlândia', 'Porto Alegre', 'Cravinhos', 'Pereiro', 'Itajaí',
       'Betim', 'Macaé', 'Goiânia', 'Palotina', 'Campo Grande', 'Serra',
       'Tijucas', 'Novo Hamburgo', 'Osasco', 'Maringá', 'Cachoeirinha',
       'Vitória', 'Mossoró', 'Sorocaba', 'Dois Irmãos', 'Arcos',
       'Jaraguá Do Sul', 'Joinville', 'Capivari', 'Várzea Grande',
       'Brasil', 'Marília', 'Aracaju', 'Pato Branco', 'Franca',
       'Rio Do Sul', 'Sumaré', 'Ibirama', 'Belo Horizonte', 'Recife',
       'Manaus', 'Embu Das Artes', 'Eldorado Do Sul', 'Viçosa',
       'São José Do Rio Preto', 'Leopoldina', 'Monte Belo', 'Londrina',
       'Salvador', 'Morrinhos', 'Vila Velha', 'Ribeirão Preto', 'Cuiabá',
       'Florianópolis', 'Belém', 'Jundiaí', 'Santo André', 'Santa Rosa',
       'Guarulhos', 'Matão', '

In [72]:
job_copy[(job_copy[['city', 'state', 'country']] != df[['city', 'state', 'country']]).any(axis=1)][['city', 'state', 'country']]

,city,state,country
23,São Paulo,None,Brasil
25,São Paulo,None,Brasil
26,São Paulo,None,Brasil
30,Rio de Janeiro,RJ,Brasil
31,São Paulo,None,Brasil
...,...,...,...
3485,None,None,Brasil
3486,None,None,Brasil
3490,Rio de Janeiro,None,Brasil
3503,None,None,Brasil


In [89]:
mask = (job_copy[['city', 'state', 'country']] != df[['city', 'state', 'country']])
# Show both DataFrames side by side where they differ
differences = pd.concat([job_copy[mask.any(axis=1)], df[mask.any(axis=1)]], axis=1, keys=['job_copy', 'df'])
differences[[('job_copy', 'city'), ('job_copy', 'state'), ('job_copy', 'country'), ('df', 'city'), ('df', 'state'), ('df', 'country')]].sample(10)

job_copy                                  df              
                      city state country                  city state country
830              São Paulo  None  Brasil             São Paulo    SP  Brasil
724                   None  None  Brasil                Brasil  None  Brasil
3351        Rio de Janeiro  None  Brasil        Rio De Janeiro  None  Brasil
2419  São José dos Pinhais    PR  Brasil  São José Dos Pinhais    PR  Brasil
3399        Rio de Janeiro  None  Brasil        Rio De Janeiro  None  Brasil
2345             São Paulo  None  Brasil             São Paulo    SP  Brasil
2027        Rio de Janeiro    RJ  Brasil        Rio De Janeiro    RJ  Brasil
1573                  None  None  Brasil                Brasil  None  Brasil
1997         Caxias do Sul    RS  Brasil         Caxias Do Sul    RS  Brasil
496                   None  None  Brasil                Brasil  None  Brasil

In [56]:
df

,job_id,work_model,keyword,scrape_date,job_title,company_name,location,num_applicants,xp_level,job_type,job_sectors,job_description,classified_job_title,post_date,city,state,country
0,4219203458,Presencial,Analista de Dados,2025-05-17,Analista de Dados e Serviço ao Cliente Junior,Cielo,"Barueri, SP",177.0,Assistente,Tempo integral,Atividades de serviços financeiros,"Job Description\nSomos mais que uma máquina, s...",Analista de Dados,03-05-2025,Barueri,SP,Brasil
1,4229491792,Presencial,Analista de Dados,2025-05-17,Analista de dados de negócios,SulAmérica,"São Paulo, SP",117.0,Pleno-sênior,Tempo integral,Seguros e previdência complementar e Serviços ...,A SulAmérica há mais de 125 anos se dedica a e...,Analista de Dados,16-05-2025,São Paulo,SP,Brasil
2,4209520797,Presencial,Analista de Dados,2025-05-17,Analista de Dados Júnior,Banco Fibra,"São Paulo, SP",163.0,Não aplicável,Tempo integral,Bancos,Somos um Banco que trabalha na busca do melhor...,Analista de Dados,19-04-2025,São Paulo,SP,Brasil
3,4201072662,Presencial,Analista de Dados,2025-05-17,"Analista de Dados - Júnior, Pleno e Sênior | R...",Capco,"São Paulo, SP",112.0,Pleno-sênior,Tempo integral,Atividades de serviços financeiros,SOBRE A CAPCO\nA Capco é uma consultoria globa...,Analista de Dados,26-04-2025,São Paulo,SP,Brasil
4,4223039814,Presencial,Analista de Dados,2025-05-17,Analista de Dados & Analytics Pleno | Riscos e...,C6 Bank,"São Paulo, SP",89.0,Assistente,Tempo integral,Bancos,Nossa área de Processos & Controles\nA área de...,Analista de Dados,10-05-2025,São Paulo,SP,Brasil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3500,4231053333,Presencial,Analista de BI,2025-05-18,Programador de Sistemas de Informação,Sebratel,"Porto Alegre, RS",NaN,Assistente,Tempo integral,Telecomunicações,"DESCRIÇÃO\nA SEBRATEL, empresa referência no s...",Outros,16-05-2025,Porto Alegre,RS,Brasil
3501,4232436842,Híbrido,Analista de BI,2025-05-18,Analista de BI (Inteligência de Mercado) - MG,Drogaria Araujo S/A,"Belo Horizonte, MG",NaN,Pleno-sênior,Tempo integral,Comércio varejista,"A Drogaria Araujo, a maior Rede de Varejo Farm...",Analista de BI,NaN,Belo Horizonte,MG,Brasil
3502,4232150473,Híbrido,Analista de BI,2025-05-18,Analista de bi business intelligence,Netvagas,"São Paulo, SP",NaN,Pleno-sênior,Tempo integral,Fornecimento e gestão de recursos humanos,"Na Connect For People, acreditamos em conectar...",Analista de BI,17-05-2025,São Paulo,SP,Brasil
3503,4232380566,Remoto,Analista de BI,2025-05-18,Analista de bi pleno,Netvagas,Brasil,NaN,Assistente,Tempo integral,Fornecimento e gestão de recursos humanos,"A FIAP é uma faculdade de tecnologia, inovação...",Analista de BI,17-05-2025,Brasil,None,Brasil
